# Install Spark

In [2]:
#If you are using  Data proc comment the file for spark instalattion.
# !sudo apt update
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# #Update spark version for install
# !wget -q  https://archive.apache.org/dist/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
# !tar xf spark-3.3.2-bin-hadoop3.tgz

# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"


# !pip install -q findspark
# !pip install pyspark

# import findspark
# findspark.init()
# findspark.find()

from pyspark.sql import DataFrame, SparkSession



spark = SparkSession \
       .builder \
       .appName("Our First Spark example") \
       .getOrCreate()

spark

In [3]:
spark = SparkSession \
       .builder \
       .appName("Our First Spark example") \
       .getOrCreate()
spark

# Download the file

In [3]:
import urllib
url = 'https://drive.google.com/file/d/1BFx9CClKTe_syBuIr5WBi5uuSkZf9wdv/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
urllib.request.urlretrieve(url, "people-with-dups.csv")


('people-with-dups.csv', <http.client.HTTPMessage at 0x7f4fc4207430>)

# Instructions

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) De-Duping Data Lab

In this exercise, we're doing ETL on a file we've received from a customer. That file contains data about people, including:

* first, middle and last names
* gender
* birth date
* Social Security number
* salary

But, as is unfortunately common in data we get from this customer, the file contains some duplicate records. Worse:

* In some of the records, the names are mixed case (e.g., "Carol"), while in others, they are uppercase (e.g., "CAROL").
* The Social Security numbers aren't consistent either. Some of them are hyphenated (e.g., "992-83-4829"), while others are missing hyphens ("992834829").

The name fields are guaranteed to match, if you disregard character case, and the birth dates will also match. The salaries will match as well,
and the Social Security Numbers *would* match if they were somehow put in the same format.

Your job is to remove the duplicate records. The specific requirements of your job are:

* Remove duplicates. It doesn't matter which record you keep; it only matters that you keep one of them.
* Standarize the field ssn with out hyphens.
* Create a colum age base on the birth date.



Finally, you will write the results as a Parquet file.


**Hint:** The initial dataset contains 10,000 records.<br/>
The de-duplicated result has 9,976 records.


# Read the file

In [34]:
#load the file into HDFS system
# Run this just is are using a data proc cluster
!hdfs dfs -put people-with-dups.csv /user/root/people-with-dups.csv
!hdfs dfs -ls /user/root/

Spark_Hands_on_googleAfrica_.ipynb  etc		media		      srv
Untitled.ipynb			    hadoop	mnt		      sys
Untitled1.ipynb			    home	opt		      tmp
Untitled2.ipynb			    lib		people-with-dups.csv  usr
bin				    lib32	proc		      var
boot				    lib64	root
copyright			    libx32	run
dev				    lost+found	sbin
Found 2 items
drwxr-xr-x   - root hadoop          0 2023-09-22 17:47 /user/root/.sparkStaging
-rw-r--r--   1 root hadoop     712779 2023-09-22 18:28 /user/root/people-with-dups.csv


In [4]:
#Read data from csv file
#Optional read the csv file from a gcp bucket
from pyspark.sql.functions import *
path = "people-with-dups.csv"

df = (spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv(path))
 

In [5]:
#see just 10 record of the data set
df.show(10)

+---------+----------+---------+------+-------------------+------+-----------+
|firstName|middleName| lastName|gender|          birthDate|salary|        ssn|
+---------+----------+---------+------+-------------------+------+-----------+
|  Emanuel|   Wallace|   Panton|     M|1988-03-04 00:00:00|101255|935-90-7627|
|   Eloisa|     Rubye|Cayouette|     F|2000-06-20 00:00:00|204031|935-89-9009|
|    Cathi|  Svetlana|    Prins|     F|2012-12-22 00:00:00| 35895|959-30-7957|
|  Mitchel|    Andres|Mozdzierz|     M|1966-05-06 00:00:00| 55108|989-27-8093|
|    Angla|     Melba|Hartzheim|     F|1938-07-26 00:00:00| 13199|935-27-4276|
|   Rachel|    Marlin|Borremans|     F|1923-02-23 00:00:00| 67070|996-41-8616|
| Catarina|  Phylicia|  Dominic|     F|1969-09-29 00:00:00|201021|999-84-8888|
|  Antione|     Randy| Hamacher|     M|2004-03-05 00:00:00|271486|917-96-3554|
| Madaline|  Shawanda| Piszczek|     F|1996-03-17 00:00:00|183944|963-87-9974|
|  Luciano|   Norbert|  Sarcone|     M|1962-12-14 00

# Processing
You can use the [SQL API reference](https://hyukjin-spark.readthedocs.io/en/latest/reference/pyspark.sql.html) for help

In [7]:
#Create new name columns with only lower case names
dedupedDF = df.withColumn('firstName_lower', lower(col('firstName'))) \
          .withColumn('middleName_lower', lower(col('middleName'))) \
          .withColumn('lastName_lower', lower(col('lastName')))

#Create new ssn column without any dashes
dedupedDF = dedupedDF.withColumn('ssn_new', regexp_replace('ssn', '-', ''))

#Create a new age field

#Drop duplicates based only on the new columns, gender, birthDate and salary
dedupedDF = dedupedDF.dropDuplicates(['firstName_lower', 'middleName_lower', 'lastName_lower', 'ssn_new', 'gender', 'birthDate', 'salary'])

#Drop the old columns
dedupedDF = dedupedDF.drop('firstName_lower', 'middleName_lower', 'lastName_lower', 'ssn')


# In a diferente DF get and show the avagre salary per year from People betwwen 20 and 55 years old.


#See everything is ok with the final df
dedupedDF.show(5)



+---------+----------+--------+------+-------------------+------+---------+
|firstName|middleName|lastName|gender|          birthDate|salary|  ssn_new|
+---------+----------+--------+------+-------------------+------+---------+
|  Rachele|     Tamar|  Aaberg|     F|1955-04-13 00:00:00| 56069|940134488|
|  Rebecca|    Bonnie|Aanderud|     F|1946-02-13 00:00:00|171207|927874203|
|   Sammie|   Gillian|  Aarant|     F|1915-06-19 00:00:00|173553|946573500|
| Bernardo|     Kasey|  Aavang|     M|1964-03-10 00:00:00|195490|983337353|
|    Luigi| Guadalupe|  Abadie|     M|1937-01-21 00:00:00| 53677|923148351|
+---------+----------+--------+------+-------------------+------+---------+
only showing top 5 rows



In [12]:
#See the number of rows


9976

# Save the file

In [8]:
#Write the processed data frame on parquet format
#Optional Write the csv file in a gcp bucket
path = "people-deduped.parquet"


In [10]:
#validate if the parque file was writed in the HDFS
!hdfs dfs -ls /user/root/people_without_dups

Found 2 items
-rw-r--r--   1 root hadoop          0 2023-09-25 20:30 /user/root/people_without_dups/_SUCCESS
-rw-r--r--   1 root hadoop     334386 2023-09-25 20:29 /user/root/people_without_dups/part-00000-620e74c5-f4c6-4a53-9f63-a0449e2fc248-c000.snappy.parquet


# Check your answer

In [14]:
finalCount = dedupedDF.count()
assert finalCount == 9976, "expected 9976 records in finalDF"